# Подбор моделей для этапа ранжирования кандидатов

### Описание
Для пакетной рекомендательной системы характерны 3 этапа работы:
1. Генерация кандидатов. На этом этапе происходит отбор сотен или тысяч объектов-кандидатов из множества объектов, которых могут быть миллиарды. Цель заключается в том, чтобы включить в этот набор макси-мально релевантные пользователю объекты и исключить нерелевантные.
2. Скоринг и ранжирование. Здесь система оценивает кандидатов, полученных на предыдущем этапе, и присваивает каждому из них баллы, основанные на их релевантности для пользователя. Т.к. количество объек-тов на этом этапе ограничено, то можно использовать больший набор при-знаков объектов, контекстные факторы, например время и геопозицию.
3. Повторное ранжирование. В рамках данного этапа проводится корректировка списка кандидатов, применяются бизнес-правила, использу-ются метрики разнообразия и новизны для уточнения позиций объектов-кандидатов в итоговом списке рекомендаций.

В рамках этой задачи выполняется подбор моделей для этапа скоринга и ранжирования.
Ограничение: использование только CPU.

### Имеющиеся данные
Социально-демографическая информация о пользователях и список их покупок - дата, название, цена, количество.
Список сгенерированных кандидатов в разрезе покупателей.

Разделение на трейн и тест: даты покупок. Самые последние покупки в тесте.
Названия датафреймов с данными: train_data, test_data, candidates.

**Состав train_data и test_data**:
```
 0   buyer_id                           object  идентификатор покупателя
 1   buyer_birth_date                   object  дата рождения покупателя
 2   buyer_sms_allowed                  int64   покупатель разрешил присылать СМС
 3   buyer_emails_allowed               int64   покупатель разрешил присылать email
 4   buyer_account_status_loyal         int64   покупатель в статусе "Лояльный" (0 или 1)
 5   buyer_account_status_unregistered  int64   покупатель в статусе "Не зарегистрирован" (0 или 1)
 6   buyer_account_status_new           int64   покупатель в статусе "Новый" (0 или 1)
 7   buyer_account_status_potential     int64   покупатель в статусе "Потенциальный" (0 или 1)
 8   buyer_account_status_lost          int64   покупатель в статусе "Потерянный" (0 или 1)
 9   buyer_account_status_sleeping      int64   покупатель в статусе "Спящий" (0 или 1)
 10  buyer_is_female                    int64   покупатель - женщина (0 или 1)
 11  buyer_age                          int64   возраст покупателя
 12  order_id                           object  идентификатор покупки
 13  order_date                         object  дата покупки
 14  order_day_of_week                  int64   день недели, в который была совершена покупка
 15  order_hour_of_day                  int64   час дня, в который была совершена покупка
 16  product_id                         object  идентификатор товара
 17  product_name                       object  название товара
 18  product_group                      object  название группы товара
 19  product_count                      float64 количество товара
 20  product_sum                        float64 сумма за товар
```

**Состав списка сгенерированных кандидатов**:
```
 0   buyer_id                           object  идентификатор покупателя
 1   product_id                         object  идентификатор товара
```


### Метрики
- NDCG@K
- Precision@K
- Recall@K
- Diversity@K
- Novelty@K
- Serendipity@K

Оптимизируем метрику NDCG@K, т.к. на этапе скоринга и ранжирования основной задачей является формирование упорядоченного по релевантности списка рекомендаций.

## Импорты и конфигурация

In [1]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Optional
from sklearn.metrics import ndcg_score
import logging
import warnings
from multiprocessing import Pool
import optuna
from typing import List, Dict, Tuple, Optional
import logging
from datetime import datetime
import gc
from catboost import CatBoost, Pool as CBPool
from tqdm import tqdm
import os
import json
import optuna
from optuna.trial import Trial


In [2]:
ORGANIZATION_ID = ""
PROCESSING_DATE = ""
RANDOM_STATE = 42
DATA_PATH = ""

In [3]:
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)

## Классы-утилиты и общие методы

In [4]:
class MetricsCalculator:
    """
    Калькулятор метрик для оценки качества рекомендаций.

    Args:
        k_values: список значений K для расчета метрик @K
    """

    def __init__(self, k_values: List[int]):
        self.k_values = sorted(k_values)

    @staticmethod
    def _calculate_ndcg_for_user(data_tuple: Tuple[str, Dict[str, np.ndarray], int]) -> Optional[float]:
        """
        Вспомогательная функция для расчета NDCG одного пользователя.
        
        Args:
            data_tuple: кортеж (user_id, user_data, k) с данными пользователя и параметром k
            
        Returns:
            Optional[float]: значение NDCG для пользователя или None в случае ошибки
        """
        user_id, user_data, k = data_tuple
        try:
            return ndcg_score(
                y_true=[user_data["ideal"]], 
                y_score=[user_data["relevance"]], 
                k=k
            )
        except Exception as e:
            logging.error(f"Ошибка при расчете NDCG для пользователя {user_id}: {e}")
            return None
        
    def _calculate_ndcg_at_k(
        self,
        k: int,
        prepared_data: Dict[str, Dict[str, np.ndarray]],
    ) -> float:
        """
        Расчет метрики NDCG@K (Normalized Discounted Cumulative Gain) для заданного значения K.

        NDCG - это метрика, которая оценивает качество ранжирования рекомендаций с учетом
        их позиции в списке. В отличие от Precision и Recall, NDCG учитывает порядок
        рекомендаций, придавая больший вес релевантным товарам на верхних позициях.

        Формула расчета:
        DCG@K = rel₁ + Σᵢ₌₂ᵏ (relᵢ / log₂(i + 1))
        NDCG@K = DCG@K / IDCG@K
        где:
        - relᵢ - релевантность i-го товара (в нашем случае 0 или 1)
        - IDCG@K - максимально возможное значение DCG@K (идеальное ранжирование)

        Args:
            k: количество рекомендаций для оценки
            prepared_data: словарь подготовленных данных для расчета NDCG
                {
                    user_id: {
                        "relevance": np.ndarray,  # вектор релевантности рекомендаций
                        "ideal": np.ndarray       # вектор идеального ранжирования
                    }
                }

        Returns:
            float: среднее значение NDCG@K по всем пользователям.
            Возвращает 0.0 в следующих случаях:
            - если нет данных для расчета
            - если все попытки расчета NDCG завершились ошибкой
            - если prepared_data пуст

        Example:
            >>> prepared_data = {
            ...     'user1': {
            ...         'relevance': np.array([1, 0, 1, 0]),
            ...         'ideal': np.array([1, 1, 0, 0])
            ...     }
            ... }
            >>> calculator._calculate_ndcg_for_k(2, prepared_data)
            0.6309  # NDCG@2 для данного примера

        Notes:
            - Особенности метрики:
            * Учитывает порядок рекомендаций
            * Нормализована на [0, 1]
            * Чувствительна к позициям релевантных товаров
            * Использует логарифмическое дисконтирование

            - Интерпретация значений:
            * 1.0 - идеальное ранжирование
            * 0.0 - наихудшее возможное ранжирование
            * Типичные значения в реальных системах: 0.3-0.7

            - Важные замечания:
            * Метод использует готовые векторы релевантности
            * Требует предварительной подготовки данных
            * Обрабатывает ошибки для каждого пользователя отдельно
            * Использует реализацию из sklearn.metrics
        """
        if not prepared_data:
            return 0.0

        # Подготавливаем данные для параллельной обработки
        data_for_parallel = [(user_id, user_data, k) for user_id, user_data in prepared_data.items()]

        # Используем контекстный менеджер для автоматического закрытия пула
        with Pool() as pool:
            # Запускаем параллельные вычисления
            results = pool.map(MetricsCalculator._calculate_ndcg_for_user, data_for_parallel)
            
            # Фильтруем None значения и считаем среднее
            valid_results = [r for r in results if r is not None]
            
            return float(np.mean(valid_results)) if valid_results else 0.0
        
    def _calculate_precision_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
        k: int,
    ) -> Optional[float]:
        """
        Расчет Precision@K для одного пользователя.

        Precision (точность) показывает, какая доля рекомендованных системой товаров
        в топ-K оказалась релевантной для пользователя. Эта метрика оценивает
        способность системы давать точные рекомендации без "шума".

        Формула расчета:
        Precision@K = |релевантные товары ∩ рекомендованные товары в топ-K| / K

        Args:
            pred_items_at_k: список первых K рекомендованных товаров
            true_items: множество фактически купленных товаров
            k: количество рекомендаций для оценки

        Returns:
            float: значение Precision@K для пользователя
            None: если нет данных для расчета

        Example:
            >>> pred_items_at_k = ['item1', 'item2']
            >>> true_items = {'item1', 'item4'}
            >>> calculator._calculate_precision_for_user(pred_items_at_k, true_items, k=2)
            0.5  # из двух рекомендованных товаров только один оказался релевантным

        Notes:
            - Высокое значение Precision означает, что большинство рекомендаций релевантны
            - Особенности метрики:
            * Всегда нормализована на K
            * При увеличении K обычно уменьшается
            * Не учитывает общее количество релевантных товаров
            * Чувствительна к порядку рекомендаций (в топ-K попадают первые K товаров)
        """
        if not pred_items_at_k:
            return None

        relevant_count = len(set(pred_items_at_k) & true_items)
        return relevant_count / k

    def _calculate_recall_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
    ) -> Optional[float]:
        """
        Расчет метрики Recall@K (полноты) для одного пользователя.

        Recall (полнота) показывает, какую долю от всех релевантных товаров пользователя
        система смогла рекомендовать в топ-K рекомендациях. Эта метрика оценивает
        способность системы находить все интересные пользователю товары.

        Формула расчета:
        Recall@K = |релевантные товары ∩ рекомендованные товары в топ-K| / |релевантные товары|

        Args:
            pred_items_at_k: список первых K рекомендованных товаров
            true_items: множество фактически купленных товаров

        Returns:
            float: значение Recall@K для пользователя
            None: если нет данных для расчета (пустой список рекомендаций или нет релевантных товаров)

        Example:
            >>> pred_items_at_k = ['item1', 'item2']
            >>> true_items = {'item1', 'item2', 'item4'}
            >>> calculator._calculate_recall_for_user(pred_items_at_k, true_items)
            0.667  # из трех релевантных товаров (item1, item2, item4)
                # в топ-2 рекомендациях найдено два (item1, item2)

        Notes:
            - Особенности метрики для одного пользователя:
            * Чувствительна к количеству релевантных товаров
            * При увеличении K обычно растет
            * Может быть низкой, если у пользователя много релевантных товаров
            * Не учитывает порядок рекомендаций в топ-K
            * Не учитывает нерелевантные рекомендации

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Возвращает None если нет релевантных товаров
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если все релевантные товары найдены
            * Равен 0.0, если не найдено ни одного релевантного товара
        """
        if not pred_items_at_k or not true_items:
            return None

        relevant_count = len(set(pred_items_at_k) & true_items)
        return relevant_count / len(true_items)

    def _calculate_diversity_for_user(
        self,
        pred_items_at_k: List[str],
        item_categories: Dict[str, str],
    ) -> Optional[float]:
        """
        Расчет метрики Diversity (разнообразия) для одного пользователя.

        Diversity (разнообразие) измеряет насколько разнообразны рекомендации с точки зрения
        категорий товаров. Высокое разнообразие означает, что система способна рекомендовать
        товары из разных категорий, а не концентрируется только на нескольких популярных
        категориях.

        Формула расчета:
        Diversity = количество уникальных категорий в рекомендациях / общее количество категорий

        Args:
            pred_items_at_k: список рекомендованных товаров
            item_categories: словарь соответствия товаров и их категорий {item_id: category_id}

        Returns:
            float: значение Diversity для пользователя
            None: если нет данных для расчета (пустой список рекомендаций или нет категорий)

        Example:
            >>> pred_items_at_k = ['item1', 'item2', 'item3']
            >>> item_categories = {
            ...     'item1': 'category1',
            ...     'item2': 'category1',
            ...     'item3': 'category2'
            ... }
            >>> calculator._calculate_diversity_for_user(pred_items_at_k, item_categories)
            0.5  # рекомендации содержат товары из 2 категорий из 4 возможных

        Notes:
            - Высокое значение разнообразия может указывать на то, что система
            предлагает пользователям широкий спектр товаров
            - Низкое значение может говорить о том, что система "зациклилась"
            на определенных категориях
            - Особенности метрики для одного пользователя:
            * Нормализована на общее количество доступных категорий
            * Не зависит от порядка рекомендаций
            * Учитывает только уникальные категории
            * Игнорирует товары без категорий

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Возвращает None если нет информации о категориях
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если рекомендации охватывают все категории
            * Равен 0.0, если все рекомендации из одной категории

        """
        if not pred_items_at_k or not item_categories:
            return None

        total_categories = len(set(item_categories.values()))
        if total_categories == 0:
            return None

        recommended_categories = {
            item_categories[item] for item in pred_items_at_k if item in item_categories
        }

        if not recommended_categories:
            return None

        return len(recommended_categories) / total_categories

    def _calculate_novelty_for_user(
        self,
        pred_items_at_k: List[str],
        user_items: set[str],
    ) -> Optional[float]:
        """
        Расчет метрики Novelty (новизны) для одного пользователя.

        Novelty (новизна) измеряет способность системы рекомендовать товары,
        которые пользователь ранее не встречал в своей истории покупок.
        Эта метрика помогает оценить, насколько система способна предлагать
        новый контент вместо того, чтобы рекомендовать только то,
        что пользователь уже покупал.

        Формула расчета:
        Novelty = количество новых товаров в рекомендациях / количество рекомендаций
        где новый товар - это товар, которого нет в истории покупок пользователя.

        Args:
            pred_items_at_k: список рекомендованных товаров
            user_items: множество товаров из истории покупок пользователя

        Returns:
            float: значение Novelty для пользователя
            None: если нет данных для расчета (пустой список рекомендаций)

        Example:
            >>> pred_items_at_k = ['item1', 'item2', 'item3']
            >>> user_items = {'item1', 'item4'}
            >>> calculator._calculate_novelty_for_user(pred_items_at_k, user_items)
            0.667  # из трех рекомендованных товаров два (item2, item3) являются новыми

        Notes:
            - Особенности метрики для одного пользователя:
            * Нормализована на количество рекомендаций
            * Не зависит от порядка рекомендаций
            * Учитывает только факт наличия/отсутствия товара в истории
            * Не учитывает популярность товаров среди других пользователей

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если все рекомендованные товары новые
            * Равен 0.0, если все рекомендованные товары уже были у пользователя
            * Пустая история покупок считается валидной (все товары будут новыми)

            - Слишком высокое значение новизны может также означать, что рекомендации
            недостаточно персонализированы или слишком случайны
        """
        if not pred_items_at_k:
            return None

        new_items_count = sum(1 for item in pred_items_at_k if item not in user_items)
        return new_items_count / len(pred_items_at_k)

    def _calculate_serendipity_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
        popular_items: set[str],
    ) -> Optional[float]:
        """
        Расчет метрики Serendipity (неожиданности) для одного пользователя.

        Serendipity (неожиданность) измеряет способность системы рекомендовать релевантные,
        но неочевидные товары. Товар считается неожиданным, если он релевантен для пользователя
        (присутствует в тестовых данных) и при этом не входит в множество популярных товаров.

        Эта метрика помогает оценить способность системы находить "скрытые жемчужины" -
        товары, которые нравятся пользователю, но не являются очевидным выбором.

        Формула расчета:
        Serendipity = количество релевантных неожиданных товаров / количество рекомендаций
        где релевантный неожиданный товар - это товар, который:
        1) есть в тестовых покупках пользователя (релевантный)
        2) не входит в множество популярных товаров (неожиданный)

        Args:
            pred_items_at_k: список рекомендованных товаров
            true_items: множество фактически купленных товаров из тестового периода
            popular_items: множество идентификаторов популярных товаров

        Returns:
            float: значение Serendipity для пользователя
            None: если нет данных для расчета (пустой список рекомендаций)

        Example:
            >>> pred_items_at_k = ['item1', 'item2', 'item3']
            >>> true_items = {'item1', 'item3'}
            >>> popular_items = {'item1', 'item2'}
            >>> calculator._calculate_serendipity_for_user(
            ...     pred_items_at_k, true_items, popular_items
            ... )
            0.333  # только item3 является релевантным и неожиданным
                # (item1 релевантный, но популярный; item2 популярный и нерелевантный)

        Notes:
            - Особенности метрики для одного пользователя:
            * Нормализована на количество рекомендаций
            * Не зависит от порядка рекомендаций
            * Учитывает как релевантность, так и популярность
            * Более строгая метрика, чем Novelty или Precision
            * Помогает оценить качество "неочевидных" рекомендаций

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если все рекомендации релевантные и непопулярные
            * Равен 0.0, если нет релевантных непопулярных рекомендаций
            * Обычно имеет более низкие значения, чем другие метрики
            * Зависит от выбранного порога популярности товаров
        """
        if not pred_items_at_k:
            return None

        serendipitous_items = sum(
            1
            for item in pred_items_at_k
            if item in true_items and item not in popular_items
        )

        return serendipitous_items / len(pred_items_at_k)

    def _prepare_data(
        self,
        recommendations: Dict[str, List[str]],
        test_data: pd.DataFrame,
    ) -> Dict[str, Dict[str, np.ndarray]]:
        """
        Подготовка данных для расчета NDCG.

        Args:
            recommendations: словарь {user_id: list of recommended items}
            test_data: тестовый датафрейм с колонками 'buyer_id' и 'product_id'

        Returns:
            Dict с подготовленными данными для каждого пользователя
        """
        true_items = test_data.groupby("buyer_id")["product_id"].agg(list).to_dict()

        all_items = sorted(
            list(set(item for items in recommendations.values() for item in items))
        )
        item_to_idx = {item: idx for idx, item in enumerate(all_items)}

        prepared_data = {}

        for user_id, pred_items in recommendations.items():
            if user_id not in true_items:
                continue

            true_set = set(true_items[user_id])
            n_items = len(all_items)

            relevance = np.zeros(n_items)
            for item in pred_items:
                if item in item_to_idx:
                    relevance[item_to_idx[item]] = 1 if item in true_set else 0

            ideal = np.zeros(n_items)
            for item in true_set:
                if item in item_to_idx:
                    ideal[item_to_idx[item]] = 1

            prepared_data[user_id] = {"relevance": relevance, "ideal": ideal}

        return prepared_data

    def calculate(
        self,
        recommendations: Dict[str, List[str]],
        train_data: pd.DataFrame,
        test_data: pd.DataFrame,
        item_categories: Optional[Dict[str, str]] = None,
    ) -> Dict[str, float]:
        """
        Расчет всех метрик

        Args:
            recommendations: словарь {user_id: list of recommended items}
            train_data: тренировочный датафрейм для расчета новизны
            test_data: тестовый датафрейм с колонками 'buyer_id' и 'product_id'
            item_categories: словарь {item_id: category_id} для расчета разнообразия

        Returns:
            Dict[str, float]: словарь с метриками NDCG@K, Precision@K, Recall@K,
            Diversity@K, Novelty@K и Serendipity@K для каждого значения K
        """
        # Подготавливаем общие данные
        prepared_data = self._prepare_data(recommendations, test_data)
        true_items = test_data.groupby("buyer_id")["product_id"].agg(set).to_dict()
        user_history = train_data.groupby("buyer_id")["product_id"].agg(set).to_dict()

        # Подготавливаем данные для serendipity
        item_counts = train_data["product_id"].value_counts()
        threshold = np.percentile(
            item_counts.values, 80
        )  # 1 - 0.2 (percentile по умолчанию)
        popular_items = set(item_counts[item_counts >= threshold].index)

        # Инициализируем словарь с метриками
        metrics = {}

        # Инициализируем списки для хранения значений метрик
        precision_values = []
        recall_values = []
        diversity_values = []
        novelty_values = []
        serendipity_values = []

        # Рассчитываем все метрики
        for k in self.k_values:
            metrics[f"ndcg_{k}"] = self._calculate_ndcg_at_k(k, prepared_data)

            for user_id, pred_items in recommendations.items():
                pred_items_at_k = pred_items[:k]
                user_id_in_true_items = user_id in true_items

                # Precision
                if user_id_in_true_items:
                    precision_value = self._calculate_precision_for_user(
                        pred_items_at_k=pred_items_at_k,
                        true_items=true_items[user_id],
                        k=k,
                    )
                    if precision_value is not None:
                        precision_values.append(precision_value)

                # Recall
                if user_id_in_true_items:
                    recall_value = self._calculate_recall_for_user(
                        pred_items_at_k=pred_items_at_k, true_items=true_items[user_id]
                    )
                    if recall_value is not None:
                        recall_values.append(recall_value)

                # Diversity
                diversity_value = self._calculate_diversity_for_user(
                    pred_items_at_k=pred_items_at_k,
                    item_categories=item_categories or {},
                )
                if diversity_value is not None:
                    diversity_values.append(diversity_value)

                # Novelty
                if user_id in user_history:
                    novelty_value = self._calculate_novelty_for_user(
                        pred_items_at_k=pred_items_at_k,
                        user_items=user_history[user_id],
                    )
                    if novelty_value is not None:
                        novelty_values.append(novelty_value)

                # Serendipity
                if user_id_in_true_items:
                    serendipity_value = self._calculate_serendipity_for_user(
                        pred_items_at_k=pred_items_at_k,
                        true_items=true_items[user_id],
                        popular_items=popular_items,
                    )
                    if serendipity_value is not None:
                        serendipity_values.append(serendipity_value)

            # Сохраняем средние значения метрик
            metrics[f"precision_{k}"] = (
                float(np.mean(precision_values)) if precision_values else 0.0
            )
            metrics[f"recall_{k}"] = (
                float(np.mean(recall_values)) if recall_values else 0.0
            )
            metrics[f"diversity_{k}"] = (
                float(np.mean(diversity_values)) if diversity_values else 0.0
            )
            metrics[f"novelty_{k}"] = (
                float(np.mean(novelty_values)) if novelty_values else 0.0
            )
            metrics[f"serendipity_{k}"] = (
                float(np.mean(serendipity_values)) if serendipity_values else 0.0
            )

        return metrics

## Бейзлайн

In [28]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [6]:
class PopularItemsRanker:
    """
    Ранжировщик на основе популярности товаров.
    """

    def __init__(self):
        self.item_scores_ = None

    def fit(
        self,
        data: pd.DataFrame,
        item_col: str = "product_id",
    ) -> "PopularItemsRanker":
        """
        Обучение ранжировщика на исторических данных.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с историческими данными о покупках
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        self : PopularItemsRanker
            Обученный объект
        """
        item_counts = data[item_col].value_counts()

        self.item_scores_ = (item_counts / item_counts.max()).to_dict()

        return self

    def recommend(
        self,
        candidates: pd.DataFrame,
        top_n: int = 10,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов на основе популярности товаров.

        Args:
        ----------
        candidates : pd.DataFrame
            Датафрейм с парами пользователь-товар для ранжирования
        top_n : int, по умолчанию 10
            Количество рекомендуемых товаров для каждого пользователя
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с ранжированными рекомендациями и их скорами
        """
        if self.item_scores_ is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit().")

        recommendations = candidates.copy()

        recommendations["score"] = recommendations[item_col].map(
            lambda x: self.item_scores_.get(x, 0.0)
        )

        recommendations = recommendations.sort_values(
            by=[user_col, "score"], ascending=[True, False]
        )

        recommendations = recommendations.groupby(user_col).head(top_n)

        return recommendations

In [29]:
logging.info("Создаем и обучаем ранжировщик")
ranker = PopularItemsRanker()
ranker.fit(train_data)

logging.info("Получаем рекомендации от ранжировщика")
ranked_recommendations = ranker.recommend(candidates, top_n=10)

ranker_recommendations = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

metrics_calculator = MetricsCalculator(k_values=[10, 100, 1000])

logging.info("Расчет метрик для рекомендаций от PopularItemsRanker")
best_metrics = metrics_calculator.calculate(
    recommendations=ranker_recommendations,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")


2025-04-18 11:21:30 - INFO - Создаем и обучаем ранжировщик
2025-04-18 11:21:30 - INFO - Получаем рекомендации от ранжировщика
2025-04-18 11:22:23 - INFO - Расчет метрик для рекомендаций от PopularItemsRanker
2025-04-18 11:24:27 - INFO - Результаты:
2025-04-18 11:24:27 - INFO - Метрики для K=10:
2025-04-18 11:24:27 - INFO - NDCG@10: 0.3458
2025-04-18 11:24:27 - INFO - Precision@10: 0.0623
2025-04-18 11:24:27 - INFO - Recall@10: 0.1476
2025-04-18 11:24:27 - INFO - Diversity@10: 0.0049
2025-04-18 11:24:27 - INFO - Novelty@10: 0.8546
2025-04-18 11:24:27 - INFO - Serendipity@10: 0.0000
2025-04-18 11:24:27 - INFO - --------------------------------
2025-04-18 11:24:27 - INFO - Метрики для K=100:
2025-04-18 11:24:27 - INFO - NDCG@100: 0.4384
2025-04-18 11:24:27 - INFO - Precision@100: 0.0342
2025-04-18 11:24:27 - INFO - Recall@100: 0.1476
2025-04-18 11:24:27 - INFO - Diversity@100: 0.0049
2025-04-18 11:24:27 - INFO - Novelty@100: 0.8546
2025-04-18 11:24:27 - INFO - Serendipity@100: 0.0000
2025

## Модель 1: CatBoost (Градиентный бустинг над решающими деревьями)

In [5]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [6]:
class CatBoostRanker:
    """
    Ранжировщик на основе CatBoost.

    Attrs:
        model: CatBoost
            Модель CatBoost для ранжирования
        feature_names: List[str]
            Список используемых признаков
        categorical_features: List[str]
            Список категориальных признаков
        numeric_fill_values_: Dict[str, float]
            Словарь для хранения значений для заполнения пропусков в числовых признаках
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        iterations: int = 1000,
        depth: int = 6,
        l2_leaf_reg: float = 3.0,
        random_seed: int = 42,
        thread_count: int = -1,
        verbose: bool = True,
    ):
        """
        Инициализация ранжировщика.

        Args:
            learning_rate: скорость обучения
            iterations: количество итераций
            depth: глубина деревьев
            l2_leaf_reg: L2 регуляризация
            random_seed: seed для воспроизводимости
            thread_count: количество потоков (-1 для использования всех)
            verbose: выводить ли прогресс обучения
        """
        self.model_params = {
            "learning_rate": learning_rate,
            "iterations": iterations,
            "depth": depth,
            "l2_leaf_reg": l2_leaf_reg,
            "random_seed": random_seed,
            "thread_count": thread_count,
            "verbose": verbose,
            "task_type": "CPU",
            "loss_function": "Logloss",
            "eval_metric": "NDCG",
            "early_stopping_rounds": 50,
        }

        self.feature_names = [
            # Социально-демографические признаки
            "buyer_sms_allowed",
            "buyer_emails_allowed",
            "buyer_account_status_loyal",
            "buyer_account_status_unregistered",
            "buyer_account_status_new",
            "buyer_account_status_potential",
            "buyer_account_status_lost",
            "buyer_account_status_sleeping",
            "buyer_is_female",
            "buyer_age",
            # Временные признаки
            "order_day_of_week",
            "order_hour_of_day",
            # Признаки покупок
            "product_count",
            "product_sum",
            # Категориальные признаки
            "product_group",
        ]

        self.categorical_features = ["product_group"]
        self.model = None
        self.numeric_fill_values_ = {}  # Добавляем инициализацию словаря

    def _prepare_features(
        self, data: pd.DataFrame, is_train: bool = True
    ) -> Tuple[pd.DataFrame, Optional[np.ndarray]]:
        """
        Подготовка признаков для обучения или предсказания.

        Args:
            data: исходный датафрейм
            is_train: флаг обучения/предсказания

        Returns:
            Tuple с подготовленными признаками и целевой переменной (если is_train=True)
        """
        features = data[self.feature_names].copy()

        for col in features.columns:
            if col in self.categorical_features:
                features[col] = features[col].fillna("unknown")
            else:
                if is_train:
                    fill_value = (
                        features[col].median() if features[col].notnull().any() else 0
                    )
                    self.numeric_fill_values_ = self.numeric_fill_values_ or {}
                    self.numeric_fill_values_[col] = fill_value
                else:
                    fill_value = self.numeric_fill_values_.get(col, 0)
                features[col] = features[col].fillna(fill_value)

        if is_train:
            target = np.ones(len(features))
            return features, target
        else:
            return features, None

    def fit(
        self,
        train_data: pd.DataFrame,
        candidates: pd.DataFrame,
        validation_size: float = 0.2,
    ) -> "CatBoostRanker":
        """
        Обучение модели ранжирования.

        Args:
            train_data: исторические данные о покупках
            candidates: датафрейм с кандидатами
            validation_size: доля данных для валидации

        Returns:
            self: обученная модель
        """
        train_data = train_data.sort_values("buyer_id")
        candidates = candidates.sort_values("buyer_id")

        X_pos, y_pos = self._prepare_features(train_data, is_train=True)
        groups_pos = train_data["buyer_id"].values

        features_df = train_data[["buyer_id", "product_id"] + self.feature_names].copy()
        features_df = features_df.drop_duplicates(subset=["buyer_id", "product_id"])

        del train_data
        gc.collect()

        chunk_size = 100000
        X_chunks = []
        groups_chunks = []
        total_negative_samples = 0

        for start_idx in tqdm(
            range(0, len(candidates), chunk_size),
            desc="Подготовка отрицательных примеров",
        ):
            end_idx = start_idx + chunk_size
            chunk = candidates.iloc[start_idx:end_idx]

            chunk_with_features = chunk.merge(
                features_df, on=["buyer_id", "product_id"], how="left"
            )

            X_chunk, _ = self._prepare_features(chunk_with_features, is_train=True)
            groups_chunk = chunk_with_features["buyer_id"].values

            X_chunks.append(X_chunk)
            groups_chunks.append(groups_chunk)
            total_negative_samples += len(X_chunk)

            del chunk_with_features, X_chunk, groups_chunk
            gc.collect()

        del features_df
        gc.collect()

        X_neg = pd.DataFrame(columns=X_pos.columns)
        groups_neg = np.empty(total_negative_samples, dtype=groups_pos.dtype)
        y_neg = np.zeros(total_negative_samples)

        current_idx = 0
        for X_chunk, groups_chunk in zip(X_chunks, groups_chunks):
            chunk_size = len(X_chunk)
            X_neg = pd.concat([X_neg, X_chunk], ignore_index=True)
            groups_neg[current_idx : current_idx + chunk_size] = groups_chunk
            current_idx += chunk_size

            del X_chunk
            gc.collect()

        del X_chunks, groups_chunks
        gc.collect()

        X = pd.concat([X_pos, X_neg], ignore_index=True)
        y = np.concatenate([y_pos, y_neg])
        groups = np.concatenate([groups_pos, groups_neg])

        del X_pos, X_neg, y_pos, y_neg, groups_pos, groups_neg
        gc.collect()

        sort_idx = np.argsort(groups)
        X = X.iloc[sort_idx].reset_index(drop=True)
        y = y[sort_idx]
        groups = groups[sort_idx]

        unique_groups = np.unique(groups)
        n_groups = len(unique_groups)
        np.random.shuffle(unique_groups)
        train_groups = set(unique_groups[: int(n_groups * (1 - validation_size))])

        train_mask = np.array([g in train_groups for g in groups])
        valid_mask = ~train_mask

        train_pool = CBPool(
            data=X[train_mask],
            label=y[train_mask],
            group_id=groups[train_mask],
            cat_features=self.categorical_features,
        )

        valid_pool = CBPool(
            data=X[valid_mask],
            label=y[valid_mask],
            group_id=groups[valid_mask],
            cat_features=self.categorical_features,
        )

        del X, y, groups, train_mask, valid_mask
        gc.collect()

        self.model = CatBoost(self.model_params)
        self.model.fit(train_pool, eval_set=valid_pool, plot=False)

        return self

    def rank(
        self, candidates: pd.DataFrame, train_data: pd.DataFrame, top_n: int = 10
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов.

        Args:
            candidates: датафрейм с кандидатами
            train_data: исторические данные для получения признаков
            top_n: количество рекомендаций для каждого пользователя

        Returns:
            DataFrame с ранжированными рекомендациями
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit().")

        candidates = candidates.sort_values("buyer_id").reset_index(drop=True)

        features_df = train_data[
            ["buyer_id", "product_id"] + self.feature_names
        ].drop_duplicates()

        candidates_with_features = candidates.merge(
            features_df, on=["buyer_id", "product_id"], how="left"
        )

        if len(candidates_with_features) != len(candidates):
            candidates_with_features = candidates_with_features.drop_duplicates(
                subset=["buyer_id", "product_id"]
            ).reset_index(drop=True)

        X_pred, _ = self._prepare_features(candidates_with_features, is_train=False)

        scores = self.model.predict(
            CBPool(data=X_pred, cat_features=self.categorical_features)
        )

        if len(scores) != len(candidates):
            raise ValueError(
                f"Количество предсказаний ({len(scores)}) не совпадает с "
                f"количеством кандидатов ({len(candidates)})"
            )

        recommendations = candidates.copy()
        recommendations["score"] = scores

        recommendations = recommendations.sort_values(
            by=["buyer_id", "score"], ascending=[True, False]
        )
        recommendations = recommendations.groupby("buyer_id").head(top_n)

        return recommendations[["buyer_id", "product_id", "score"]]

    def save_model(
        self, model_path: str, format: str = "cbm", export_parameters: bool = True
    ) -> None:
        """
        Сохранение модели и её параметров.

        Args:
            model_path: путь для сохранения модели
            format: формат сохранения ('cbm' или 'json')
            export_parameters: сохранять ли дополнительные параметры модели
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit()")

        os.makedirs(os.path.dirname(model_path), exist_ok=True)

        self.model.save_model(model_path, format=format, pool=None)

        if export_parameters:
            parameters = {
                "model_params": self.model_params,
                "feature_names": self.feature_names,
                "categorical_features": self.categorical_features,
                "numeric_fill_values": self.numeric_fill_values_,
            }

            params_path = f"{os.path.splitext(model_path)[0]}_params.json"

            with open(params_path, "w", encoding="utf-8") as f:
                json.dump(parameters, f, ensure_ascii=False, indent=2)

    @classmethod
    def load_model(
        cls, model_path: str, load_parameters: bool = True
    ) -> "CatBoostRanker":
        """
        Загрузка сохранённой модели и её параметров.

        Args:
            model_path: путь к сохранённой модели
            load_parameters: загружать ли дополнительные параметры модели

        Returns:
            CatBoostRanker: загруженная модель
        """
        ranker = cls()

        ranker.model = CatBoost()
        ranker.model.load_model(model_path)

        if load_parameters:
            params_path = f"{os.path.splitext(model_path)[0]}_params.json"

            if os.path.exists(params_path):
                with open(params_path, "r", encoding="utf-8") as f:
                    parameters = json.load(f)

                ranker.model_params = parameters["model_params"]
                ranker.feature_names = parameters["feature_names"]
                ranker.categorical_features = parameters["categorical_features"]
                ranker.numeric_fill_values_ = parameters["numeric_fill_values"]
            else:
                logging.warning(
                    f"Файл с параметрами {params_path} не найден. "
                    "Загружена только модель без дополнительных параметров."
                )

        return ranker

In [12]:
def optimize_catboost_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    candidates: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 10,
    timeout: int = 7200,
) -> Tuple[Dict, Dict[str, float]]:
    """
    Оптимизация параметров CatBoostRanker с помощью Optuna.

    Args:
        train_data: тренировочные данные
        test_data: тестовые данные
        candidates: кандидаты для ранжирования
        n_trials: количество итераций оптимизации
        timeout: максимальное время оптимизации в секундах

    Returns:
        Tuple[Dict, Dict[str, float]]:
            - Лучшие параметры
            - Значения метрик на лучших параметрах
    """

    def objective(trial: Trial) -> float:
        model_params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'iterations': 1000,
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
            'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
            'early_stopping_rounds': 50,
            'task_type': 'CPU',
            'thread_count': -1,
            'verbose': False,
            'eval_metric': 'NDCG',
            'loss_function': 'Logloss'
        }

        ranker = CatBoostRanker(
            learning_rate=model_params['learning_rate'],
            iterations=model_params['iterations'],
            depth=model_params['depth'],
            l2_leaf_reg=model_params['l2_leaf_reg'],
            random_seed=42,
            thread_count=model_params['thread_count'],
            verbose=model_params['verbose']
        )

        ranker.model_params.update(model_params)

        ranker.fit(train_data, candidates)

        recommendations = ranker.rank(
            candidates=candidates,
            train_data=train_data,
            top_n=max([10, 100, 1000]),  # для расчета всех метрик
        )

        recommendations_dict = (
            recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
        )

        metrics = metrics_calculator.calculate(
            recommendations=recommendations_dict,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        target_k = max(metrics_calculator.k_values)
        return metrics[f"ndcg_{target_k}"]

    study = optuna.create_study(
        direction="maximize",
        study_name=f'catboost_ranker_optimization_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    )

    logging.info("Начало оптимизации параметров...")
    study.optimize(
        objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True
    )

    best_params = study.best_params

    best_model_params = {
        'learning_rate': best_params['learning_rate'],
        'iterations': 1000,
        'depth': best_params['depth'],
        'l2_leaf_reg': best_params['l2_leaf_reg'],
        'random_seed': 42,
        'thread_count': -1,
        'verbose': True,
        'task_type': 'CPU',
        'loss_function': 'Logloss',
        'eval_metric': 'NDCG',
        'early_stopping_rounds': 50,
        'random_strength': best_params['random_strength'],
        'bagging_temperature': best_params['bagging_temperature'],
        'leaf_estimation_iterations': best_params['leaf_estimation_iterations']
    }

    logging.info("Обучение модели на лучших параметрах...")
    best_ranker = CatBoostRanker(
        learning_rate=best_model_params['learning_rate'],
        iterations=best_model_params['iterations'],
        depth=best_model_params['depth'],
        l2_leaf_reg=best_model_params['l2_leaf_reg'],
        random_seed=42,
        thread_count=best_model_params['thread_count'],
        verbose=best_model_params['verbose']
    )
    best_ranker.model_params.update(best_model_params)
    best_ranker.fit(train_data, candidates)

    recommendations = best_ranker.rank(
        candidates=candidates, train_data=train_data, top_n=max([10, 100, 1000])
    )

    recommendations_dict = (
        recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
    )

    final_metrics = metrics_calculator.calculate(
        recommendations=recommendations_dict,
        train_data=train_data,
        test_data=test_data,
        item_categories=item_categories,
    )

    logging.info("\nЛучшие параметры:")
    for param_name, param_value in best_params.items():
        logging.info(f"{param_name}: {param_value}")

    logging.info("\nМетрики на лучших параметрах:")
    for metric_name, metric_value in final_metrics.items():
        logging.info(f"{metric_name}: {metric_value:.4f}")

    return best_params, final_metrics

In [14]:
if __name__ == "__main__":
    n_samples = 100
    # buyer_ids = test_data["buyer_id"].unique()[:n_samples]
    buyer_ids = test_data["buyer_id"].unique()

    temp_candidates = candidates[candidates["buyer_id"].isin(buyer_ids)].copy()

    product_ids = temp_candidates["product_id"].unique()
    temp_train_data = train_data[
        (train_data["buyer_id"].isin(buyer_ids))
        & (train_data["product_id"].isin(product_ids))
    ].copy()

    temp_test_data = test_data[
        (test_data["buyer_id"].isin(buyer_ids))
        & (test_data["product_id"].isin(product_ids))
    ].copy()

    metrics_calculator = MetricsCalculator([10, 100, 1000])

    best_params, best_metrics = optimize_catboost_parameters(
        train_data=temp_train_data,
        test_data=temp_test_data,
        candidates=temp_candidates,
        metrics_calculator=metrics_calculator,
        n_trials=10,
        timeout=7200,
    )

    logging.info(f"Лучшие параметры: {best_params}")
    logging.info("Результаты:")
    for k in metrics_calculator.k_values:
        logging.info(f"Метрики для K={k}:")
        logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
        logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
        logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
        logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
        logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
        logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
        logging.info("--------------------------------")

[I 2025-04-21 12:00:34,560] A new study created in memory with name: catboost_ranker_optimization_20250421_120034
2025-04-21 12:00:34 - INFO - Начало оптимизации параметров...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [05:16<00:00,  1.90it/s]
IOStream.flush timed out
  0%|          | 0/10 [1:28:41<?, ?it/s]

[I 2025-04-21 13:29:15,605] Trial 0 finished with value: 0.8317919801478025 and parameters: {'learning_rate': 0.004684283500572371, 'depth': 6, 'l2_leaf_reg': 5.36104374072008e-06, 'random_strength': 0.007652648646042903, 'bagging_temperature': 0.9487648394691982, 'leaf_estimation_iterations': 9}. Best is trial 0 with value: 0.8317919801478025.


Best trial: 0. Best value: 0.831792:  10%|█         | 1/10 [2:37:40<13:18:09, 5321.05s/it, 5321.05/7200 seconds]

[I 2025-04-21 14:38:15,522] Trial 1 finished with value: 0.8317919801478025 and parameters: {'learning_rate': 0.02388179353529231, 'depth': 5, 'l2_leaf_reg': 4.8339398459122034e-08, 'random_strength': 0.0031822125482293437, 'bagging_temperature': 0.5517318159431245, 'leaf_estimation_iterations': 2}. Best is trial 0 with value: 0.8317919801478025.


Best trial: 0. Best value: 0.831792:  20%|██        | 2/10 [2:37:40<10:30:43, 4730.48s/it, 9460.96/7200 seconds]
2025-04-21 14:38:15 - INFO - Обучение модели на лучших параметрах...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [04:13<00:00,  2.37it/s]


0:	test: 0.7768302	best: 0.7768302 (0)	total: 6.48s	remaining: 1h 47m 55s
1:	test: 0.7768302	best: 0.7768302 (0)	total: 12.9s	remaining: 1h 47m 21s
2:	test: 0.7768302	best: 0.7768302 (0)	total: 19s	remaining: 1h 45m 24s
3:	test: 0.7768302	best: 0.7768302 (0)	total: 22.2s	remaining: 1h 32m 9s
4:	test: 0.7768302	best: 0.7768302 (0)	total: 25.6s	remaining: 1h 24m 45s
5:	test: 0.7768302	best: 0.7768302 (0)	total: 28.8s	remaining: 1h 19m 32s
6:	test: 0.7768302	best: 0.7768302 (0)	total: 32.1s	remaining: 1h 15m 53s
7:	test: 0.7768302	best: 0.7768302 (0)	total: 35.3s	remaining: 1h 12m 59s
8:	test: 0.7768302	best: 0.7768302 (0)	total: 38.6s	remaining: 1h 10m 54s
9:	test: 0.7768302	best: 0.7768302 (0)	total: 41.9s	remaining: 1h 9m 12s
10:	test: 0.7768302	best: 0.7768302 (0)	total: 45.2s	remaining: 1h 7m 42s
11:	test: 0.7768302	best: 0.7768302 (0)	total: 48.6s	remaining: 1h 6m 37s
12:	test: 0.7768302	best: 0.7768302 (0)	total: 51.8s	remaining: 1h 5m 34s
13:	test: 0.7768302	best: 0.7768302 (0)	to

2025-04-21 15:27:34 - INFO - 
Лучшие параметры:
2025-04-21 15:27:34 - INFO - learning_rate: 0.004684283500572371
2025-04-21 15:27:34 - INFO - depth: 6
2025-04-21 15:27:34 - INFO - l2_leaf_reg: 5.36104374072008e-06
2025-04-21 15:27:34 - INFO - random_strength: 0.007652648646042903
2025-04-21 15:27:34 - INFO - bagging_temperature: 0.9487648394691982
2025-04-21 15:27:34 - INFO - leaf_estimation_iterations: 9
2025-04-21 15:27:34 - INFO - 
Метрики на лучших параметрах:
2025-04-21 15:27:34 - INFO - ndcg_10: 0.8393
2025-04-21 15:27:34 - INFO - precision_10: 0.1210
2025-04-21 15:27:34 - INFO - recall_10: 0.3156
2025-04-21 15:27:34 - INFO - diversity_10: 0.0052
2025-04-21 15:27:34 - INFO - novelty_10: 0.3672
2025-04-21 15:27:34 - INFO - serendipity_10: 0.0039
2025-04-21 15:27:34 - INFO - ndcg_100: 0.8272
2025-04-21 15:27:34 - INFO - precision_100: 0.0697
2025-04-21 15:27:34 - INFO - recall_100: 0.3538
2025-04-21 15:27:34 - INFO - diversity_100: 0.0225
2025-04-21 15:27:34 - INFO - novelty_100: 0

## Сохранение лучших моделей

In [ ]:
logging.info("Обучение модели CatBoostRanker...")
model_params = {
    "learning_rate": 0.004684283500572371,
    "iterations": 1000,
    "depth": 6,
    "l2_leaf_reg": 5.36104374072008e-06,
    "random_seed": 42,
    "thread_count": -1,
    "verbose": True,
    "task_type": "CPU",
    "loss_function": "Logloss",
    "eval_metric": "NDCG",
    "early_stopping_rounds": 50,
    "random_strength": 0.007652648646042903,
    "bagging_temperature": 0.9487648394691982,
    "leaf_estimation_iterations": 9,
}

cat_boost_ranker = CatBoostRanker(
    learning_rate=model_params["learning_rate"],
    iterations=model_params["iterations"],
    depth=model_params["depth"],
    l2_leaf_reg=model_params["l2_leaf_reg"],
    random_seed=42,
    thread_count=model_params["thread_count"],
    verbose=model_params["verbose"],
)
cat_boost_ranker.model_params.update(model_params)
cat_boost_ranker.fit(train_data, candidates)

logging.info("Сохранение модели CatBoostRanker...")
model_path = (
    f"{DATA_PATH}/models/cat_boost_ranker_{ORGANIZATION_ID}_{PROCESSING_DATE}.cbm"
)
cat_boost_ranker.save_model(model_path)

logging.info("Загрузка модели CatBoostRanker...")
cat_boost_ranker = CatBoostRanker.load_model(model_path)

logging.info("Генерация финальных рекомендаций...")
ranked_recommendations = cat_boost_ranker.rank(
    candidates=candidates, train_data=train_data, top_n=1000
)

recommendations_dict = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

logging.info("Расчет финальных метрик...")
metrics_calculator = MetricsCalculator([10, 100, 1000])
final_metrics = metrics_calculator.calculate(
    recommendations=recommendations_dict,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {final_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-22 13:23:03 - INFO - Обучение модели CatBoostRanker...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [05:40<00:00,  1.77it/s]


0:	test: 0.8775637	best: 0.8775637 (0)	total: 7.58s	remaining: 2h 6m 9s
1:	test: 0.8775637	best: 0.8775637 (0)	total: 13.4s	remaining: 1h 51m 29s
2:	test: 0.8775637	best: 0.8775637 (0)	total: 15.7s	remaining: 1h 26m 57s
3:	test: 0.8775637	best: 0.8775637 (0)	total: 18.2s	remaining: 1h 15m 35s
4:	test: 0.8775637	best: 0.8775637 (0)	total: 20.5s	remaining: 1h 8m
5:	test: 0.8775637	best: 0.8775637 (0)	total: 26.1s	remaining: 1h 12m 8s
6:	test: 0.8775637	best: 0.8775637 (0)	total: 28.5s	remaining: 1h 7m 29s
7:	test: 0.8775637	best: 0.8775637 (0)	total: 30.8s	remaining: 1h 3m 44s
8:	test: 0.8846712	best: 0.8846712 (8)	total: 34.2s	remaining: 1h 2m 48s
9:	test: 0.8846712	best: 0.8846712 (8)	total: 36.4s	remaining: 1h 1s
10:	test: 0.8846442	best: 0.8846712 (8)	total: 39.8s	remaining: 59m 36s
11:	test: 0.8846431	best: 0.8846712 (8)	total: 42.1s	remaining: 57m 42s
12:	test: 0.8847303	best: 0.8847303 (12)	total: 44.3s	remaining: 56m 2s
13:	test: 0.8846362	best: 0.8847303 (12)	total: 46.5s	remain

Training has stopped (degenerate solution on iteration 513, probably too small l2-regularization, try to increase it)
2025-04-22 14:05:26 - INFO - Сохранение модели CatBoostRanker...
2025-04-22 14:05:26 - INFO - Загрузка модели CatBoostRanker...
2025-04-22 14:05:27 - INFO - Генерация финальных рекомендаций...
IOStream.flush timed out
2025-04-22 14:08:33 - INFO - Расчет финальных метрик...
2025-04-22 14:11:44 - INFO - Результаты:
2025-04-22 14:11:44 - INFO - Метрики для K=10:
2025-04-22 14:11:44 - INFO - NDCG@10: 0.8367
2025-04-22 14:11:44 - INFO - Precision@10: 0.1193
2025-04-22 14:11:44 - INFO - Recall@10: 0.3068
2025-04-22 14:11:44 - INFO - Diversity@10: 0.0052
2025-04-22 14:11:44 - INFO - Novelty@10: 0.3679
2025-04-22 14:11:44 - INFO - Serendipity@10: 0.0013
2025-04-22 14:11:44 - INFO - --------------------------------
2025-04-22 14:11:44 - INFO - Метрики для K=100:
2025-04-22 14:11:44 - INFO - NDCG@100: 0.8246
2025-04-22 14:11:44 - INFO - Precision@100: 0.0688
2025-04-22 14:11:44 -

## Сравнение моделей

### Лучшие гиперпараметры моделей

**CatBoost (Градиентный бустинг над решающими деревьями)**:
```json
{
    'learning_rate': 0.004684283500572371, 
    'depth': 6, 
    'l2_leaf_reg': 5.36104374072008e-06,
    'random_strength': 0.007652648646042903,
    'bagging_temperature': 0.9487648394691982,
    'leaf_estimation_iterations': 9
}
```

### Метрики работы моделей

| Метрика  | CatBoost  |
|---|---|
|  Время обучения |  40 мин. |
|  Время получения предсказаний |  3 мин. |
|  NDCG@10 |  0.8367 |
|  NDCG@100 |  0.8246 |
|  NDCG@1000 |  0.8291 |
|  Precision@10 |  0.1193 |
|  Precision@100 |  0.0688 |
|  Precision@1000 |  0.0471 |
|  Recall@10 |  0.3068 |
|  Recall@100 |  0.3453 |
|  Recall@1000 |  0.4839 |
|  Diversity@10 |  0.0052 |
|  Diversity@100 |  0.0225 |
|  Diversity@1000 |  0.0777 |
|  Novelty@10 |  0.3679 |
|  Novelty@100 |  0.6367 |
|  Novelty@1000 |  0.7546 |
|  Serendipity@10 |  0.0013 |
|  Serendipity@100 |  0.0007 |
|  Serendipity@1000 |  0.0005 |

**Сравнение временных затрат производилось на одинаковых датасетах и одинаковом оборудовании на платформе Intel Ice Lake с 32 vCPU и 256 Гб RAM**
